In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm_notebook



structure = pd.read_csv("course-217-structure.csv").sort_values(by=["module_position", "lesson_position", "step_position"])
data = pd.read_csv("course-217-events.csv")

In [2]:
structure.head()

,course_id,module_id,module_position,lesson_id,lesson_position,step_id,step_position,step_type,step_cost
219,217,635,1,13140,1,38518,1,text,0
221,217,635,1,13140,1,38521,2,text,0
220,217,635,1,13140,1,38519,3,choice,0
229,217,635,1,13140,1,38529,4,code,1
227,217,635,1,13140,1,38527,5,text,0


In [3]:
step_ids = structure.step_id.values
steps = defaultdict(set)

In [4]:
def get_next_step(step_id):
    if not step_id == 83966:
        return step_ids[np.where(step_id == step_ids)[0][0] + 1]

In [5]:
get_next_step(38518)

38521

In [15]:
res = defaultdict(set)

for user_id in tqdm_notebook(data.user_id.unique()):
    # проходим по всем юзерам и находим их степы
    user_steps = data[data.user_id == user_id].step_id.values
    # теперь составляем словарь степ - все его пользователи
    for i, step in enumerate(user_steps):
        steps[step].add(user_id)
        next_step = get_next_step(step)
        try:
            index_a = np.where(step == user_steps[:i])[0][0]
            index_b = i - np.where(next_step == user_steps[:i][::-1])[0][0] - 1
            if index_a < index_b:
                res[step].add(user_id)           
        except:
            pass
        

/Library/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:12: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [16]:
len(res[41684])

561

In [17]:
a = {i: len(res[i]) / len(steps[i]) for i in steps.keys()}

In [21]:
a[41684]

0.7460106382978723

In [19]:
sorted([k for k in a], key=lambda k: a[k], reverse=True)[:10]

[41684, 41604, 41097, 41481, 42593, 38872, 41686, 39735, 39740, 39717]

In [20]:
",".join(map(str, sorted([k for k in a], key=lambda k: a[k], reverse=True)[:10]))

'41684,41604,41097,41481,42593,38872,41686,39735,39740,39717'